The idea is to build a k-d tree for the training set X_train to speed up the nearest neighbour computations for a nearest neighbor classifier.

In [1]:
%matplotlib inline
import math
import copy
import numpy
import time

In [2]:
class Neighbours():
    """ Class that can be used to keep track of the 
    k nearest neighbours computed. It also conducts
    the "brute-force" phase that takes place in the
    leaves of a k-d tree (see function 'add').
    """

    def __init__(self, k):
        
        self.k = k 
        
        self._neighbors = [(None, None, float("inf")) for i in range(self.k)]
        
    def get_dists_indices(self):
        
        indices = [neigh[1] for neigh in self._neighbors]
        dists = [neigh[2] for neigh in self._neighbors]
        
        return numpy.array(dists), numpy.array(indices)
    
    def add(self, points, indices, query):
        
        for i in range(len(points)):
            
            p = points[i]
            idx = indices[i]
            dist = self._distance(p, query)
            
            self._neighbors.append([p,idx,dist])
            self._neighbors = sorted(self._neighbors, key=lambda n: n[2])
            self._neighbors = self._neighbors[:self.k]

    def get_max_dist(self):
        
        return self._neighbors[-1][2]
        
    def _distance(self, x, y):
        
        dist = ((x-y)**2).sum()
        return math.sqrt(dist)
    
class Node():
    """ Class that represents a single node of
    a k-d tree. If both 'left' and 'right' are 
    None, then the node is a leaf. The local 
    variables 'points' and 'indices' are used
    to store the points/indices assigned to a 
    leaf.
    
    Otherwise, it is an internal node that 
    stores the median (splitting hyperplane)
    """
    
    def __init__(self, 
                 left, 
                 right, 
                 median=None, 
                 points=None, 
                 indices=None):

        self.left = left
        self.right = right
        self.median = median
        self.points = points
        self.indices = indices
        
class KDTree():
    
    def __init__(self, leaf_size=30):
        """ Instantiates a k-d tree.
        
        Parameters
        ----------
        leaf_size : int, default 30
            The leaf size, i.e., the maximal 
            number of points stored in a leaf 
            of the k-d tree.
        """
        
        self.leaf_size = leaf_size
        
    def fit(self, X):
        """
        
        Parameters
        ----------
        X : array-like of shape (n, d)
            A Numpy array containing n data 
            points each having d features                
        """
        
        # remember dimension for which the tree was built
        self._dim = len(X[0])
        
        # generate a list of the "original" indices that
        # are processed in a similar way as the points; 
        # this is needed in order to obtain the indices
        # of the neighbours compuated for a query.
        original_indices = numpy.array(range(len(X)))
        
        # build tree recursively
        self._root = self._build_tree(copy.deepcopy(X),
                                      original_indices, 
                                      depth=0)     
        
    def query(self, X, k=1, max_leaves=None, alpha=1.0):
        """ Computes the k nearest neighbors for each 
        point in X.
        
        Parameters
        ----------
        X : array-like of shape (n, d)
            A Numpy array containing n data 
            points each having d features
        k : int, default 1
            The number of nearest neighbours to 
            be computed
            
        Returns
        -------
        dists, indices : arrays of shape (n, k)
            Two arrays containing, for each query point,
            the distances and the associated indices of
            its k nearest neighbors w.r.t. the points
            used for building the tree.
        """
        self.alpha = alpha
        
        
        if self._root is None:  
            raise Exception("Tree not fitted yet!")
            
        if len(X[0]) != self._dim:
            raise Exception("Tree was fitted for points of dimension: {}".format(self._dim))
        
        # initialize two empty arrays that will be used to 
        # store the distances and the associated indices
        dists = numpy.empty((len(X), k), dtype=numpy.float64)
        indices = numpy.empty((len(X), k), dtype=numpy.int32)
        
        
        # iterate over all query points
        for i in range(len(X)):
            
            # set max-leaves
            self.max_leaves=max_leaves
            # initialize the neighbours object, which
            # will keep track of the nearest neighbours
            neighbours = Neighbours(k)
            
            # start recursive search
            self._recursive_search(self._root, 
                                   X[i], 
                                   k, 
                                   depth=0, 
                                   neighbours=neighbours)
            
            # get the final distances and indices for 
            # the current query and store them at 
            # position i in the arrays dists and indices 
            dists_query, indices_query = neighbours.get_dists_indices()
            dists[i,:] = dists_query
            indices[i,:] = indices_query
                
        return dists, indices
    
    def _build_tree(self, pts, indices, depth):
        """ Builds a k-d tree for the points given in pts. Since
        we are also interested in the indidces afterwards, we also
        keep track of the (original) indices.
        
        This code is similar to the pseudocode given on 
        slides 27-29 of L3_LSDA.pdf
        """
        
        # if only self.leaf_size points are left, stop
        # the recursion and generate a leaf node
        if len(pts) <= self.leaf_size: 
            
            return Node(left=None, 
                        right=None, 
                        points=pts, 
                        indices=indices)
        
        # select axis
        axis = depth % self._dim
        
        # sort the points w.r.t. dimension 'axis';
        # also sort the indices accordingly
        partition = pts[:,axis].argsort()
        pts = pts[partition]
        indices = indices[partition]
        
        # compute splitting index and median value
        split_idx = math.floor(len(pts) / 2)
        if len(pts) % 2 == 1:
            median = pts[split_idx, axis]
        else:
            median = 0.5 * (pts[split_idx, axis] + pts[split_idx+1, axis])
        
        # build trees for children recursively ...
        lefttree = self._build_tree(pts[:split_idx,:], indices[:split_idx], depth+1)
        righttree = self._build_tree(pts[split_idx:,:], indices[split_idx:], depth+1)
        
        # return node storing all the relevant information
        return Node(left=lefttree, right=righttree, median=median)

    def _recursive_search(self, node, query, k, depth, neighbours):

        if self.max_leaves == 0 and self.max_leaves != None:
            return
        
        if (node.left == None and node.right==None):
            neighbours.add(node.points, node.indices, query)
            if self.max_leaves != None:
                self.max_leaves -= 1
            return
        # axis to be checked (same order as during construction)
        axis = depth % self._dim

        # select next subtree candidate
        if query[axis] < node.median:
            first = node.left
            second = node.right
        else:
            first = node.right
            second = node.left

        # check first subtree
        self._recursive_search(first, query, k, depth+1, neighbours)

        # while going up again (to the root): check if we 
        # still have to search in the second subtree! 
        if abs(node.median - query[axis]) < neighbours.get_max_dist()/self.alpha:
            self._recursive_search(second, query, k, depth+1, neighbours)

In [3]:
from scipy.stats import mode
import time
import numpy as np

class NearestNeighborClassifier(object):
    
    def __init__(self, n_neighbors=3, leaf_size=20, max_leaves=None, alpha=1.0):
        self.n_neighbors = n_neighbors
        self.leaf_size = leaf_size
        self.tree = KDTree(leaf_size=leaf_size)
        self.max_leaves = max_leaves
        self.alpha=alpha
        
    def fit(self, X, y):
        # fit with training data
        self.tree.fit(X)
        self.train = y
    
    def predict(self, X):
        start = time.process_time()
        dists, indices = self.tree.query(X,k=1,max_leaves=self.max_leaves,alpha=self.alpha)
        elapsed = time.process_time() - start
        print("Time needed in seconds:", elapsed)
        preds = []
        for index in range(len(X)):
            preds.append(np.argmax(np.bincount(self.train[indices[index]])))
        return np.array(preds)

In [4]:
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split

# get covtype data set; only make use of the first 10 features
covtype = fetch_covtype()
X, y = covtype.data, covtype.target
X = X[:,:10]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001, random_state=0)
print("Number of training instances: {}".format(X_train.shape[0]))
print("Number of test instances: {}".format(X_test.shape[0]))
print("Number of features: {}".format(X_train.shape[1]))

Number of training instances: 580430
Number of test instances: 582
Number of features: 10


In [5]:
# instantiate nearest neighbor classifier and fit/train model
model = NearestNeighborClassifier(n_neighbors=3)
model.fit(X_train, y_train)

In [6]:
# Subtask 1.
from sklearn.metrics import accuracy_score
preds = model.predict(X_test)
acc = accuracy_score(y_test, preds)
print("Final test accuracy: {}".format(acc))

Time needed in seconds: 21.845542
Final test accuracy: 0.9690721649484536


In [7]:
# Subtask 2.
model = NearestNeighborClassifier(n_neighbors=3,max_leaves=10)
model.fit(X_train, y_train)
from sklearn.metrics import accuracy_score
preds = model.predict(X_test)
acc = accuracy_score(y_test, preds)
print("Final test accuracy: {}".format(acc))

Time needed in seconds: 0.9807890000000015
Final test accuracy: 0.9415807560137457


In [8]:
# Subtask 3.
model = NearestNeighborClassifier(n_neighbors=3,alpha=2.0)
model.fit(X_train, y_train)
from sklearn.metrics import accuracy_score
preds = model.predict(X_test)
acc = accuracy_score(y_test, preds)
print("Final test accuracy: {}".format(acc))

Time needed in seconds: 7.842159000000002
Final test accuracy: 0.9656357388316151
